In [1]:
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os
import optuna
import json
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, TensorDataset
import torch.optim.lr_scheduler as lr_scheduler
from pathlib import Path
import seaborn as sns

# Fix OpenMP runtime duplicate initialization error
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

c:\Users\bsjun\miniforge3\envs\NN_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Load data
df = pd.read_csv('BMED_data_v5.csv')

# Prepare data for all experiments
X_batches = []
S_batches = []
Y_batches = []
times_list = []

max_points = max(len(df[df['exp'] == exp_id]) for exp_id in df['exp'].unique())

In [6]:
for exp_id in df['exp'].unique():
    df_exp = df[df['exp'] == exp_id]
    
    X = df_exp[['T','V','E','Ci','Ki']].iloc[0]
    S = df_exp[['NF_LA','NA_LA','NF_K','NB_K','VF','VA','VB']].iloc[0]
    Y = df_exp[['NF_LA','NA_LA','NF_K','NB_K','VF','VA','VB']].values
    times = df_exp['t'].values
    
    # Pad Y with the last values to match max_points
    if len(Y) < max_points:
        pad_length = max_points - len(Y)
        Y = np.pad(Y, ((0, pad_length), (0, 0)), mode='edge')
        times = np.pad(times, (0, pad_length), mode='edge')
    
    X_batches.append(torch.FloatTensor(X.values))
    S_batches.append(torch.FloatTensor(S.values))
    Y_batches.append(torch.FloatTensor(Y))
    times_list.append(times)

In [18]:
# Convert to batches
X_batch = torch.stack(X_batches)
S_batch = torch.stack(S_batches)
Y_batch = torch.stack(Y_batches)

In [21]:
Y_batch

tensor([[[ 5.0000e-01,  0.0000e+00,  1.0000e+00,  ...,  1.0000e+00,
           1.0000e+00,  1.0000e+00],
         [ 4.9166e-01,  8.3435e-03,  1.0124e+00,  ...,  1.0200e+00,
           9.9000e-01,  9.8000e-01],
         [ 4.6336e-01,  3.6636e-02,  8.2681e-01,  ...,  1.0200e+00,
           9.9000e-01,  9.8000e-01],
         ...,
         [ 1.0178e-01,  3.9822e-01,  2.0147e-01,  ...,  9.1000e-01,
           9.8000e-01,  1.1200e+00],
         [ 1.0178e-01,  3.9822e-01,  2.0147e-01,  ...,  9.1000e-01,
           9.8000e-01,  1.1200e+00],
         [ 1.0178e-01,  3.9822e-01,  2.0147e-01,  ...,  9.1000e-01,
           9.8000e-01,  1.1200e+00]],

        [[ 5.0000e-01,  0.0000e+00,  1.0000e+00,  ...,  1.0000e+00,
           1.0000e+00,  1.0000e+00],
         [ 5.0314e-01, -3.1369e-03,  9.9587e-01,  ...,  1.0400e+00,
           9.8000e-01,  9.7000e-01],
         [ 5.1203e-01, -1.2032e-02,  1.0871e+00,  ...,  1.0900e+00,
           9.8000e-01,  9.6000e-01],
         ...,
         [ 1.9763e-01,  3

In [5]:
# Set sequence length to Y data length
seq_length = Y.shape[0]

In [6]:
# Prepare training data
X_batch = X.unsqueeze(0).repeat(1, seq_length, 1)  # [1, seq_length, x_size]
S_batch = S.unsqueeze(0)  # [1, s_size]
Y_batch = Y.unsqueeze(0)  # [1, seq_length, s_size]

In [7]:
class BMEDLSTM(nn.Module):
    def __init__(self, x_size, s_size, hidden_size, num_layers=2):
        super(BMEDLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.s_size = s_size
        
        self.lstm = nn.LSTM(x_size + s_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, s_size)
        
    def forward(self, x, s0, dt, max_time):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(x.device)
        
        # 초기 상태 설정
        current_s = s0.unsqueeze(1)  # [batch_size, 1, s_size]
        outputs = []
        cal_times = []
        cal_t = 0.0
        
        # dt 간격으로 계산 진행
        while cal_t <= max_time:
            # 현재 입력 x와 상태 s 결합
            combined_input = torch.cat([x[:, 0:1], current_s], dim=2)  # 모든 시간에서 같은 x 사용
            
            # LSTM 순전파
            lstm_out, (h0, c0) = self.lstm(combined_input, (h0, c0))
            step_output = self.fc(lstm_out)
            
            # 상태 업데이트 (S(n+1) = S(n) + output*dt)
            current_s = current_s + step_output * dt
            
            outputs.append(current_s)
            cal_times.append(cal_t)
            cal_t += dt
        
        # 모든 출력을 시퀀스로 결합
        outputs = torch.cat(outputs, dim=1)  # [batch_size, num_steps, s_size]
        cal_times = torch.tensor(cal_times).to(x.device)
        
        return outputs, cal_times

In [8]:
# Model parameter settings
x_size = X.shape[0]
s_size = S.shape[0]
hidden_size = 32
dt = 0.01
max_time = times[-1]

In [9]:
# Initialize model
model = BMEDLSTM(x_size, s_size, hidden_size)

In [10]:
# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [11]:
# Training
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    outputs, cal_times = model(X_batch, S_batch, dt, max_time)
    
    # Find closest calculation results to actual data times
    loss = 0
    for i, t in enumerate(times):
        # Find index of closest time
        idx = torch.abs(cal_times - t).argmin()
        loss += criterion(outputs[:, idx:idx+1], Y_batch[:, i:i+1])
    
    # Backward pass
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.3144
Epoch [20/100], Loss: 0.0458
Epoch [30/100], Loss: 0.0459
Epoch [40/100], Loss: 0.0236
Epoch [50/100], Loss: 0.0239
Epoch [60/100], Loss: 0.0213
Epoch [70/100], Loss: 0.0210
Epoch [80/100], Loss: 0.0205
Epoch [90/100], Loss: 0.0201
Epoch [100/100], Loss: 0.0199


In [ ]:
# Prediction
model.eval()
with torch.no_grad():
    predictions, cal_times = model(X_batch, S_batch, dt, max_time)
    print("\nPrediction Results:")
    print(predictions[0])


Prediction Results:
tensor([[ 4.9901e-01, -3.0024e-04,  1.0001e+00,  ...,  9.9968e-01,
          1.0011e+00,  1.0012e+00],
        [ 4.9813e-01, -5.7878e-04,  9.9981e-01,  ...,  9.9956e-01,
          1.0018e+00,  1.0022e+00],
        [ 4.9733e-01, -7.8765e-04,  9.9924e-01,  ...,  9.9958e-01,
          1.0022e+00,  1.0031e+00],
        ...,
        [ 2.0340e-01,  3.1039e-01,  2.3304e-01,  ...,  9.2477e-01,
          9.8915e-01,  1.0612e+00],
        [ 2.0283e-01,  3.1103e-01,  2.3147e-01,  ...,  9.2456e-01,
          9.8910e-01,  1.0613e+00],
        [ 2.0226e-01,  3.1168e-01,  2.2990e-01,  ...,  9.2435e-01,
          9.8905e-01,  1.0615e+00]])


: 

In [ ]:

# Visualization
plt.figure(figsize=(15, 10))
y_names = ['NF_LA', 'NA_LA', 'NF_K', 'NB_K', 'VF', 'VA', 'VB']  # Column names of Y data
for i in range(len(y_names)):
    plt.subplot(3, 3, i+1)
    # Plot experimental data as scatter points
    plt.scatter(times, Y_batch[0, :, i].numpy(), c='b', label='Experimental', alpha=0.6)
    # Plot model predictions as continuous line
    plt.plot(cal_times.numpy(), predictions[0, :, i].numpy(), 'r-', label='Model Prediction', linewidth=1.5)
    plt.title(f'{y_names[i]}')
    plt.xlabel('Time (t)')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)

plt.tight_layout()
plt.show()